In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
from bs4 import BeautifulSoup
import lxml

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



**Since there is not any open source for London neighbourhood coordinates, needed file self created with using the website https://latitudelongitude.org/map-address-latlong <br>The file inserted as dataframe below.**

In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_4f9851849bb645bcbe52a19900245aef = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='*********************',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='*********************'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_4f9851849bb645bcbe52a19900245aef.get_object(Bucket='githubproject-donotdelete-pr-*********************',Key='london_postalcodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

london_pcode = pd.read_csv(body)
london_pcode.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,E1,Eastern,Eastern Head district,51.507351,-0.127758
1,E2,Eastern,Bethnal Green,51.531103,-0.048080
2,E3,Eastern,Bow,51.532089,-0.030820
3,E4,Eastern,Chingford,51.632067,0.007363
4,E5,Eastern,Clapton,51.567230,-0.061190


In [3]:
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London, United Kingdom are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London, United Kingdom are 51.5073219, -0.1276474.


In [4]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(london_pcode['Latitude'], london_pcode['Longitude'], london_pcode['Borough'], london_pcode['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
map_london

**Let's simplify the above map and segment and cluster only the neighborhoods in South Western to locate our restaurant in more popular location. So let's slice the original dataframe and create a new dataframe of the South Western district data.**

In [5]:
western_data = london_pcode[london_pcode['Borough'] == 'South Western'].reset_index(drop=True)
western_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,SW1,South Western,South Western Head district,51.507351,-0.127758
1,SW2,South Western,Brixton,51.461279,-0.115615
2,SW3,South Western,Chelsea,51.486943,-0.170037
3,SW4,South Western,Clapham,51.463698,-0.136305
4,SW5,South Western,Earls Court,51.490331,-0.195842
5,SW6,South Western,Fulham,51.477343,-0.201682
6,SW7,South Western,South Kensington,51.495329,-0.178991
7,SW8,South Western,South Lambeth,51.485381,-0.123321
8,SW9,South Western,Stockwell,51.471593,-0.121664
9,SW10,South Western,West Brompton,51.483494,-0.196802


In [6]:
address = 'South Western, London, United Kingdom'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of South Western are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of South Western are 51.3911804, -0.298542518020966.


**As we did with all of London, let's visualize at South Western the neighborhoods in it.**

In [7]:
# create map of South Western using latitude and longitude values
map_western = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(western_data['Latitude'], western_data['Longitude'], western_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_western)  
    
map_western

**Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.**

In [8]:
CLIENT_ID = '*********************' # your Foursquare ID
CLIENT_SECRET = '*********************' # your Foursquare Secret
VERSION = '20181009' 

**Let's explore the first neighborhood in our dataframe.<br>Get the neighborhood's name.**

In [9]:
western_data.loc[0, 'Neighbourhood']

'South Western Head district'

**Get the neighborhood's latitude and longitude values.**

In [10]:
neighborhood_latitude = western_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = western_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = western_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of South Western Head district are 51.507351, -0.12775799999999998.


**Now, let's get the top 100 venues that are in South Western Head district within a radius of 3000 meters.<br>First, let's create the GET request URL. Name your URL url.**

In [33]:
LIMIT = 100 
radius = 3000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


**Send the GET request and examine the results**

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d9f41e7be7078002ccd67c5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'London',
  'headerFullLocation': 'London',
  'headerLocationGranularity': 'city',
  'totalResults': 237,
  'suggestedBounds': {'ne': {'lat': 51.53435102700003,
    'lng': -0.08445947975350589},
   'sw': {'lat': 51.48035097299997, 'lng': -0.1710565202464941}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ac518cdf964a520e6a520e3',
       'name': 'National Gallery',
       'location': {'address': 'Trafalgar Sq',
        'lat': 51.50887601013219,
        'lng': -0.1284778118133545,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.50887601013219,
      

**Let's create a function *get_category_type* to all the neighborhoods in South Western**

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude','Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [16]:
western_venues = getNearbyVenues(names=western_data['Neighbourhood'],latitudes=western_data['Latitude'],longitudes=western_data['Longitude'])

South Western Head district
Brixton
Chelsea
Clapham
Earls Court
Fulham
South Kensington
South Lambeth
Stockwell
West Brompton


In [17]:
print('There are {} uniques categories.'.format(len(western_venues['Venue Category'].unique())))

There are 158 uniques categories.


**Analyze Each Neighborhood**

In [18]:
# one hot encoding
western_onehot = pd.get_dummies(western_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
western_onehot['Neighbourhood'] = western_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [western_onehot.columns[-1]] + list(western_onehot.columns[:-1])
western_onehot = western_onehot[fixed_columns]
western_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Fountain,French Restaurant,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Okonomiyaki Restaurant,Opera House,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Restaurant,Rock Climbing Spot,Roof Deck,Russian Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Plaza,Skate Park,Soccer Stadium,Social Club,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,South Western Head district,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,South Western Head district,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,South Western Head district,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,South Western Head district,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,South Western Head district,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**And let's examine the new dataframe size.**

In [19]:
western_onehot.shape

(1000, 159)

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [20]:
western_grouped = western_onehot.groupby('Neighbourhood').mean().reset_index()
western_grouped

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Fountain,French Restaurant,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Okonomiyaki Restaurant,Opera House,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Restaurant,Rock Climbing Spot,Roof Deck,Russian Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Plaza,Skate Park,Soccer Stadium,Social Club,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Brixton,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.04,0.02,0.01,0.05,0.03,0.00,0.00,0.00,0.00,0.04,0.09,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.04,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.02,0.02,0.01,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.01,0.01,0.07,0.01,0.00,0.03,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.03,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.02
1,Chelsea,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.01,0.00,0.01,0.00,0.03,0.00,0.01,0.01,0.00,0.00,0.02,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.02,0.00,0.01,0.01,0.00,0.00,0.00,0.02,0.03,0.01,0.02,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.06,0.00,0.04,0.03,0.01,0.00,0.07,0.00,0.04,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.03,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.01,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.02
2,Clapham,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.00,0.02,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.02,0.02,0.01,0.02,0.02,0.00,0.00,0.00,

**Let's confirm the new size**

In [21]:
western_grouped.shape

(10, 159)

**Let's print each neighborhood along with the top 10 most common venues**

In [22]:
num_top_venues = 10

for hood in western_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = western_grouped[western_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brixton----
                  venue  freq
0           Coffee Shop  0.09
1                   Pub  0.07
2                  Café  0.05
3                  Park  0.05
4               Brewery  0.04
5          Cocktail Bar  0.04
6                Market  0.04
7           Pizza Place  0.03
8      Tapas Restaurant  0.03
9  Caribbean Restaurant  0.03


----Chelsea----
                 venue  freq
0   Italian Restaurant  0.07
1                Hotel  0.06
2  Japanese Restaurant  0.04
3       Ice Cream Shop  0.04
4          Pizza Place  0.03
5           Restaurant  0.03
6    Indian Restaurant  0.03
7                 Café  0.03
8       Science Museum  0.03
9               Garden  0.03


----Clapham----
                   venue  freq
0                    Pub  0.10
1            Coffee Shop  0.07
2            Pizza Place  0.05
3                   Park  0.04
4                 Market  0.04
5  Portuguese Restaurant  0.03
6     Italian Restaurant  0.03
7             Restaurant  0.03
8      Indian Restau

**Let's put that into a pandas dataframe<br>First, let's write a function to sort the venues in descending order.**

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Now let's create the new dataframe and display the top 10 venues for each neighborhood.**

In [24]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = western_grouped['Neighbourhood']

for ind in np.arange(western_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(western_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brixton,Coffee Shop,Pub,Park,Café,Market,Brewery,Cocktail Bar,Caribbean Restaurant,Pizza Place,Restaurant
1,Chelsea,Italian Restaurant,Hotel,Japanese Restaurant,Ice Cream Shop,Restaurant,Pizza Place,Indian Restaurant,Garden,Café,Science Museum
2,Clapham,Pub,Coffee Shop,Pizza Place,Park,Market,Portuguese Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,French Restaurant
3,Earls Court,Café,Pizza Place,Pub,Italian Restaurant,Hotel,French Restaurant,Restaurant,Park,Science Museum,Indian Restaurant
4,Fulham,Café,Pub,Park,Coffee Shop,Pizza Place,Italian Restaurant,Soccer Stadium,Bakery,Gastropub,French Restaurant
5,South Kensington,Hotel,Indian Restaurant,Garden,Japanese Restaurant,French Restaurant,Park,Italian Restaurant,Ice Cream Shop,Science Museum,Restaurant
6,South Lambeth,Theater,Hotel,Park,Plaza,Art Gallery,Garden,Coffee Shop,Art Museum,Deli / Bodega,Café
7,South Western Head district,Hotel,Steakhouse,Clothing Store,Theater,Ice Cream Shop,Indian Restaurant,Bookstore,Art Museum,Bakery,Movie Theater
8,Stockwell,Café,Pub,Coffee Shop,Pizza Place,Park,Market,Brewery,Cocktail Bar,Cricket Ground,Restaurant
9,West Brompton,Café,Italian Restaurant,Pub,Bakery,Pizza Place,Park,Hotel,Yoga Studio,Soccer Stadium,Restaurant


**Cluster Neighborhoods** <br>
Run k-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5
western_grouped_clustering = western_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(western_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 2, 0, 3, 4, 1, 2], dtype=int32)

**Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
western_merged = western_data

# merge western_grouped with western_data to add latitude/longitude for each neighborhood
western_merged = western_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
western_merged 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,SW1,South Western,South Western Head district,51.507351,-0.127758,4,Hotel,Steakhouse,Clothing Store,Theater,Ice Cream Shop,Indian Restaurant,Bookstore,Art Museum,Bakery,Movie Theater
1,SW2,South Western,Brixton,51.461279,-0.115615,1,Coffee Shop,Pub,Park,Café,Market,Brewery,Cocktail Bar,Caribbean Restaurant,Pizza Place,Restaurant
2,SW3,South Western,Chelsea,51.486943,-0.170037,0,Italian Restaurant,Hotel,Japanese Restaurant,Ice Cream Shop,Restaurant,Pizza Place,Indian Restaurant,Garden,Café,Science Museum
3,SW4,South Western,Clapham,51.463698,-0.136305,1,Pub,Coffee Shop,Pizza Place,Park,Market,Portuguese Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,French Restaurant
4,SW5,South Western,Earls Court,51.490331,-0.195842,2,Café,Pizza Place,Pub,Italian Restaurant,Hotel,French Restaurant,Restaurant,Park,Science Museum,Indian Restaurant
5,SW6,South Western,Fulham,51.477343,-0.201682,2,Café,Pub,Park,Coffee Shop,Pizza Place,Italian Restaurant,Soccer Stadium,Bakery,Gastropub,French Restaurant
6,SW7,South Western,South Kensington,51.495329,-0.178991,0,Hotel,Indian Restaurant,Garden,Japanese Restaurant,French Restaurant,Park,Italian Restaurant,Ice Cream Shop,Science Museum,Restaurant
7,SW8,South Western,South Lambeth,51.485381,-0.123321,3,Theater,Hotel,Park,Plaza,Art Gallery,Garden,Coffee Shop,Art Museum,Deli / Bodega,Café
8,SW9,South Western,Stockwell,51.471593,-0.121664,1,Café,Pub,Coffee Shop,Pizza Place,Park,Market,Brewery,Cocktail Bar,Cricket Ground,Restaurant
9,SW10,South Western,West Brompton,51.483494,-0.196802,2,Café,Italian Restaurant,Pub,Bakery,Pizza Place,Park,Hotel,Yoga Studio,Soccer Stadium,Restaurant


**Finally, let's visualize the resulting clusters**

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(western_merged['Latitude'], western_merged['Longitude'], western_merged['Neighbourhood'], western_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)     
map_clusters

**Examine Clusters** <br>
We can now examine each cluster and determine the discriminating venue categories that distinguish each cluster.

**CLUSTER1**

In [28]:
western_merged.loc[western_merged['Cluster Labels'] == 0, western_merged.columns[[1] + list(range(5, western_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,South Western,0,Italian Restaurant,Hotel,Japanese Restaurant,Ice Cream Shop,Restaurant,Pizza Place,Indian Restaurant,Garden,Café,Science Museum
6,South Western,0,Hotel,Indian Restaurant,Garden,Japanese Restaurant,French Restaurant,Park,Italian Restaurant,Ice Cream Shop,Science Museum,Restaurant


**CLUSTER2**

In [29]:
western_merged.loc[western_merged['Cluster Labels'] == 1, western_merged.columns[[1] + list(range(5, western_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,South Western,1,Coffee Shop,Pub,Park,Café,Market,Brewery,Cocktail Bar,Caribbean Restaurant,Pizza Place,Restaurant
3,South Western,1,Pub,Coffee Shop,Pizza Place,Park,Market,Portuguese Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,French Restaurant
8,South Western,1,Café,Pub,Coffee Shop,Pizza Place,Park,Market,Brewery,Cocktail Bar,Cricket Ground,Restaurant


**CLUSTER3**

In [30]:
western_merged.loc[western_merged['Cluster Labels'] == 2, western_merged.columns[[1] + list(range(5, western_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,South Western,2,Café,Pizza Place,Pub,Italian Restaurant,Hotel,French Restaurant,Restaurant,Park,Science Museum,Indian Restaurant
5,South Western,2,Café,Pub,Park,Coffee Shop,Pizza Place,Italian Restaurant,Soccer Stadium,Bakery,Gastropub,French Restaurant
9,South Western,2,Café,Italian Restaurant,Pub,Bakery,Pizza Place,Park,Hotel,Yoga Studio,Soccer Stadium,Restaurant


**CLUSTER4**

In [31]:
western_merged.loc[western_merged['Cluster Labels'] == 3, western_merged.columns[[1] + list(range(5, western_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,South Western,3,Theater,Hotel,Park,Plaza,Art Gallery,Garden,Coffee Shop,Art Museum,Deli / Bodega,Café


**CLUSTER5**

In [32]:
western_merged.loc[western_merged['Cluster Labels'] == 4, western_merged.columns[[1] + list(range(5, western_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,South Western,4,Hotel,Steakhouse,Clothing Store,Theater,Ice Cream Shop,Indian Restaurant,Bookstore,Art Museum,Bakery,Movie Theater


**Cluster3 which is the neighnourhood named South Lambeth is the best suitable location for opening a restaurant for; <br> 1.Since there is only a cafe and a Deli/Bodega place which are not enough to eat properly.<br>2.South London is the southern part of London, England. Situated south of the River Thames, it includes the historic districts of Southwark, Lambeth, Bankside and Greenwich. This makes the area popular for the tourists and aside from locals, restaurant can make a lot profit from tourists.<br>3.South London contains a very extensive overground rail network and all of London’s trams operate within the area. So it really easy to reach from anyplace of London<br>4.The residents of the area tend to be young, transient and highly mobile. Given the industrial aesthetic, young professionals are most likely to be attracted to the area. Property prices are high and therefore pull in a largely affluent population.<br>5.The neighbourhood is situated in close proximity to Brixton, once edgy and shady but now vibrant and packed with great bars and restaurants. The afro-Caribbean heritage of Brixton has given the area its buzz and fantastic nightlife. Daytime in South Lambeth can be spent at Battersea Park, a large green space directly on the Thames. It’s usually filled with families and their canine friends on the weekends.<br>6.Despite Lambeth’s reputation as a rowdy neighbourhood, the area is going through a huge regeneration. The Battersea Power Station is a catalyst for the changes in the area. More families are purchasing property, and foreign investment is pouring in. The area benefits from two rail stations and has great access to west London by Vauxhall Bridge and the Chelsea Bridge Road.**